### 1️⃣ Importaciones y configuración

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

BASE_DIR = os.path.abspath(os.path.join(os.path.dirname("__file__"), "..", ".."))
OUTPUT_DIR = os.path.join(BASE_DIR, "output")

TRAIN_CSV = os.path.join(OUTPUT_DIR, "train_processed.csv")
VAL_CSV   = os.path.join(OUTPUT_DIR, "val_processed.csv")
TEST_CSV  = os.path.join(OUTPUT_DIR, "test_processed.csv")

sys.path.insert(0, BASE_DIR)

import sys
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from src.part_2.main import run_training


def calculate_metrics(y_true, y_pred, name="Model"):
    """Calculate and print comprehensive metrics"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    metrics = {
        'Model': name,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'MAPE': mape,
    }
    
    return metrics


def study_l2_regularization(TRAIN_CSV, CSV_VAL, CSV_TEST, activation="sigmoid"):
    """
    Study 1: L2 Regularization
    Test different L2 lambda values to find optimal regularization strength
    """
    print("\n" + "="*80)
    print("STUDY 1: L2 REGULARIZATION (WEIGHT DECAY)")
    print("="*80)
    print("\nL2 regularization adds a penalty term: Loss = MSE + (λ/2) × Σ(w²)")
    print("This prevents overfitting by penalizing large weights.\n")
    
    # Different L2 lambda values to test
    l2_values = [0.0, 0.0001, 0.001, 0.01, 0.1]
    results = []
    
    for l2_lambda in l2_values:
        print(f"\n{'─'*80}")
        print(f"Testing L2_lambda = {l2_lambda}")
        print(f"{'─'*80}")
        
        # Run training with specific L2 value
        nn, pid_test, y_test, preds_test, trainer = run_training(
            activation=activation,
            verbose=False,
            epochs=200,
            CSV_TRAIN=TRAIN_CSV,
            CSV_VAL=CSV_VAL,
            CSV_TEST=CSV_TEST,
            l2_lambda=l2_lambda,
            dropout_rate=0.0,  # No dropout for this study
        )
        
        # Calculate metrics
        metrics = calculate_metrics(y_test, preds_test, f"L2={l2_lambda}")
        
        # Get training history
        loss_df = trainer.loss_epochs()
        final_train_error = loss_df['Train Error'].iloc[-1]
        final_val_error = loss_df['Val Error'].iloc[-1]
        train_val_gap = abs(final_val_error - final_train_error)
        
        metrics['Final_Train_Error'] = final_train_error
        metrics['Final_Val_Error'] = final_val_error
        metrics['Train_Val_Gap'] = train_val_gap
        metrics['L2_Lambda'] = l2_lambda
        
        results.append(metrics)
        
        # Print results
        print(f"\nResults:")
        print(f"  RMSE:           ${metrics['RMSE']:,.2f}")
        print(f"  MAE:            ${metrics['MAE']:,.2f}")
        print(f"  R²:             {metrics['R2']:.4f}")
        print(f"  MAPE:           {metrics['MAPE']:.2f}%")
        print(f"  Train Error:    {final_train_error:.6f}")
        print(f"  Val Error:      {final_val_error:.6f}")
        print(f"  Train-Val Gap:  {train_val_gap:.6f}")
    
    # Create summary DataFrame
    results_df = pd.DataFrame(results)
    
    print("\n" + "="*80)
    print("L2 REGULARIZATION - SUMMARY TABLE")
    print("="*80)
    print(results_df[['L2_Lambda', 'RMSE', 'MAE', 'R2', 'MAPE', 'Train_Val_Gap']].to_string(index=False))
    
    # Find best model
    best_idx = results_df['RMSE'].idxmin()
    best_lambda = results_df.loc[best_idx, 'L2_Lambda']
    
    print(f"\n🏆 BEST L2 LAMBDA: {best_lambda}")
    print(f"   RMSE: ${results_df.loc[best_idx, 'RMSE']:,.2f}")
    print(f"   R²:   {results_df.loc[best_idx, 'R2']:.4f}")
    print(f"   Train-Val Gap: {results_df.loc[best_idx, 'Train_Val_Gap']:.6f}")
    
    # Analysis
    print("\n" + "─"*80)
    print("ANALYSIS:")
    baseline_rmse = results_df[results_df['L2_Lambda'] == 0.0]['RMSE'].values[0]
    best_rmse = results_df.loc[best_idx, 'RMSE']
    improvement = ((baseline_rmse - best_rmse) / baseline_rmse) * 100
    
    print(f"  • Baseline (no L2) RMSE: ${baseline_rmse:,.2f}")
    print(f"  • Best L2 RMSE:          ${best_rmse:,.2f}")
    print(f"  • Improvement:           {improvement:.2f}%")
    
    baseline_gap = results_df[results_df['L2_Lambda'] == 0.0]['Train_Val_Gap'].values[0]
    best_gap = results_df.loc[best_idx, 'Train_Val_Gap']
    gap_reduction = ((baseline_gap - best_gap) / baseline_gap) * 100
    print(f"  • Train-Val Gap reduction: {gap_reduction:.2f}%")
    print("─"*80)
    
    # Save results
    results_df.to_csv("l2_regularization_results.csv", index=False)
    print("\n✓ Results saved to: l2_regularization_results.csv")
    
    return results_df


def study_dropout(TRAIN_CSV, CSV_VAL, CSV_TEST, activation="sigmoid"):
    """
    Study 2: Dropout Regularization
    Test different dropout rates to find optimal value
    """
    print("\n\n" + "="*80)
    print("STUDY 2: DROPOUT REGULARIZATION")
    print("="*80)
    print("\nDropout randomly deactivates neurons during training.")
    print("This prevents co-adaptation and acts as an ensemble method.\n")
    
    # Different dropout rates to test
    dropout_rates = [0.0, 0.1, 0.2, 0.3, 0.5]
    results = []
    
    # Use a deeper architecture for dropout (works better with deeper networks)
    deeper_layers = [277, 64, 32, 16, 1]
    
    for dropout_rate in dropout_rates:
        print(f"\n{'─'*80}")
        print(f"Testing Dropout Rate = {dropout_rate}")
        print(f"{'─'*80}")
        
        # Run training with specific dropout rate
        nn, pid_test, y_test, preds_test, trainer = run_training(
            activation=activation,
            verbose=False,
            epochs=200,
            CSV_TRAIN=TRAIN_CSV,
            CSV_VAL=CSV_VAL,
            CSV_TEST=CSV_TEST,
            layers=deeper_layers,  # Use deeper network
            lr=0.01,  # Slightly lower LR for dropout
            l2_lambda=0.0,  # No L2 for this study
            dropout_rate=dropout_rate,
        )
        
        # Calculate metrics
        metrics = calculate_metrics(y_test, preds_test, f"Dropout={dropout_rate}")
        
        # Get training history
        loss_df = trainer.loss_epochs()
        final_train_error = loss_df['Train Error'].iloc[-1]
        final_val_error = loss_df['Val Error'].iloc[-1]
        train_val_gap = abs(final_val_error - final_train_error)
        
        
        metrics['Final_Train_Error'] = final_train_error
        metrics['Final_Val_Error'] = final_val_error
        metrics['Train_Val_Gap'] = train_val_gap
        metrics['Dropout_Rate'] = dropout_rate
        
        results.append(metrics)
        
        # Print results
        print(f"\nResults:")
        print(f"  RMSE:           ${metrics['RMSE']:,.2f}")
        print(f"  MAE:            ${metrics['MAE']:,.2f}")
        print(f"  R²:             {metrics['R2']:.4f}")
        print(f"  MAPE:           {metrics['MAPE']:.2f}%")
        print(f"  Train Error:    {final_train_error:.6f}")
        print(f"  Val Error:      {final_val_error:.6f}")
        print(f"  Train-Val Gap:  {train_val_gap:.6f}")
    
    # Create summary DataFrame
    results_df = pd.DataFrame(results)
    
    print("\n" + "="*80)
    print("DROPOUT REGULARIZATION - SUMMARY TABLE")
    print("="*80)
    print(results_df[['Dropout_Rate', 'RMSE', 'MAE', 'R2', 'MAPE', 'Train_Val_Gap']].to_string(index=False))
    
    # Find best model
    best_idx = results_df['RMSE'].idxmin()
    best_dropout = results_df.loc[best_idx, 'Dropout_Rate']
    
    print(f"\n🏆 BEST DROPOUT RATE: {best_dropout}")
    print(f"   RMSE: ${results_df.loc[best_idx, 'RMSE']:,.2f}")
    print(f"   R²:   {results_df.loc[best_idx, 'R2']:.4f}")
    print(f"   Train-Val Gap: {results_df.loc[best_idx, 'Train_Val_Gap']:.6f}")
    
    # Analysis
    print("\n" + "─"*80)
    print("ANALYSIS:")
    baseline_rmse = results_df[results_df['Dropout_Rate'] == 0.0]['RMSE'].values[0]
    best_rmse = results_df.loc[best_idx, 'RMSE']
    improvement = ((baseline_rmse - best_rmse) / baseline_rmse) * 100
    
    print(f"  • Baseline (no dropout) RMSE: ${baseline_rmse:,.2f}")
    print(f"  • Best Dropout RMSE:           ${best_rmse:,.2f}")
    print(f"  • Improvement:                 {improvement:.2f}%")
    
    baseline_gap = results_df[results_df['Dropout_Rate'] == 0.0]['Train_Val_Gap'].values[0]
    best_gap = results_df.loc[best_idx, 'Train_Val_Gap']
    gap_reduction = ((baseline_gap - best_gap) / baseline_gap) * 100
    print(f"  • Train-Val Gap reduction:     {gap_reduction:.2f}%")
    print("─"*80)
    
    # Save results
    results_df.to_csv("dropout_regularization_results.csv", index=False)
    print("\n✓ Results saved to: dropout_regularization_results.csv")
    
    return results_df


def study_combined_regularization(TRAIN_CSV, CSV_VAL, CSV_TEST, activation="sigmoid"):
    """
    Study 3: Combined L2 + Dropout
    Test combinations of both regularization techniques
    """
    print("\n\n" + "="*80)
    print("STUDY 3: COMBINED L2 + DROPOUT REGULARIZATION")
    print("="*80)
    print("\nCombining both techniques can leverage their complementary strengths:")
    print("  • L2: Penalizes large weights")
    print("  • Dropout: Prevents neuron co-adaptation\n")
    
    # Test different combinations
    combinations = [
        (0.001, 0.1, "Light regularization"),
        (0.001, 0.2, "Moderate regularization"),
        (0.01, 0.2, "Strong L2 + moderate dropout"),
        (0.001, 0.3, "Light L2 + strong dropout"),
    ]
    
    results = []
    deeper_layers = [277, 64, 32, 16, 1]
    
    for l2_lambda, dropout_rate, description in combinations:
        print(f"\n{'─'*80}")
        print(f"Testing: {description}")
        print(f"  L2 Lambda = {l2_lambda}, Dropout Rate = {dropout_rate}")
        print(f"{'─'*80}")
        
        # Run training
        nn, pid_test, y_test, preds_test, trainer = run_training(
            activation=activation,
            verbose=False,
            epochs=200,
            CSV_TRAIN=TRAIN_CSV,
            CSV_VAL=CSV_VAL,
            CSV_TEST=CSV_TEST,
            layers=deeper_layers,
            lr=0.001,
            l2_lambda=l2_lambda,
            dropout_rate=dropout_rate,
        )
        
        # Calculate metrics
        metrics = calculate_metrics(y_test, preds_test, description)
        
        # Get training history
        loss_df = trainer.loss_epochs()
        final_train_error = loss_df['Train Error'].iloc[-1]
        final_val_error = loss_df['Val Error'].iloc[-1]
        train_val_gap = abs(final_val_error - final_train_error)
        
        metrics['Final_Train_Error'] = final_train_error
        metrics['Final_Val_Error'] = final_val_error
        metrics['Train_Val_Gap'] = train_val_gap
        metrics['L2_Lambda'] = l2_lambda
        metrics['Dropout_Rate'] = dropout_rate
        metrics['Description'] = description
        
        results.append(metrics)
        
        # Print results
        print(f"\nResults:")
        print(f"  RMSE:           ${metrics['RMSE']:,.2f}")
        print(f"  MAE:            ${metrics['MAE']:,.2f}")
        print(f"  R²:             {metrics['R2']:.4f}")
        print(f"  MAPE:           {metrics['MAPE']:.2f}%")
        print(f"  Train Error:    {final_train_error:.6f}")
        print(f"  Val Error:      {final_val_error:.6f}")
        print(f"  Train-Val Gap:  {train_val_gap:.6f}")
    
    # Create summary DataFrame
    results_df = pd.DataFrame(results)
    
    print("\n" + "="*80)
    print("COMBINED REGULARIZATION - SUMMARY TABLE")
    print("="*80)
    print(results_df[['Description', 'L2_Lambda', 'Dropout_Rate', 'RMSE', 'R2', 'Train_Val_Gap']].to_string(index=False))
    
    # Find best model
    best_idx = results_df['RMSE'].idxmin()
    
    print(f"\n🏆 BEST COMBINATION:")
    print(f"   {results_df.loc[best_idx, 'Description']}")
    print(f"   L2 Lambda: {results_df.loc[best_idx, 'L2_Lambda']}")
    print(f"   Dropout Rate: {results_df.loc[best_idx, 'Dropout_Rate']}")
    print(f"   RMSE: ${results_df.loc[best_idx, 'RMSE']:,.2f}")
    print(f"   R²:   {results_df.loc[best_idx, 'R2']:.4f}")
    
    # Save results
    results_df.to_csv("combined_regularization_results.csv", index=False)
    print("\n✓ Results saved to: combined_regularization_results.csv")
    
    return results_df

In [ ]:
def calculate_metrics(y_true, y_pred, name="Model"):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    return {
        'Model': name,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'MAPE': mape,
    }


In [ ]:
l2_results = study_l2_regularization(
    TRAIN_CSV,
    VAL_CSV,
    TEST_CSV,
    activation="sigmoid"
)



STUDY 1: L2 REGULARIZATION (WEIGHT DECAY)

L2 regularization adds a penalty term: Loss = MSE + (λ/2) × Σ(w²)
This prevents overfitting by penalizing large weights.


────────────────────────────────────────────────────────────────────────────────
Testing L2_lambda = 0.0
────────────────────────────────────────────────────────────────────────────────
Epoch 20/200 - Train Error: 0.006990, Val Error: 0.006112
Epoch 40/200 - Train Error: 0.006881, Val Error: 0.006012
Epoch 60/200 - Train Error: 0.006309, Val Error: 0.005486
Epoch 80/200 - Train Error: 0.004256, Val Error: 0.003624
Epoch 100/200 - Train Error: 0.001957, Val Error: 0.001735
Epoch 120/200 - Train Error: 0.001446, Val Error: 0.001389
Epoch 140/200 - Train Error: 0.001225, Val Error: 0.001201
Epoch 160/200 - Train Error: 0.001074, Val Error: 0.001057
Epoch 180/200 - Train Error: 0.000967, Val Error: 0.000956
Epoch 200/200 - Train Error: 0.000890, Val Error: 0.000889

Results:
  RMSE:           $27,205.52
  MAE:            $19,

In [7]:
dropout_results = study_dropout(
    TRAIN_CSV,
    VAL_CSV,
    TEST_CSV,
    activation="sigmoid"
)




STUDY 2: DROPOUT REGULARIZATION

Dropout randomly deactivates neurons during training.
This prevents co-adaptation and acts as an ensemble method.


────────────────────────────────────────────────────────────────────────────────
Testing Dropout Rate = 0.0
────────────────────────────────────────────────────────────────────────────────
Epoch 20/200 - Train Error: 0.007066, Val Error: 0.006215
Epoch 40/200 - Train Error: 0.007062, Val Error: 0.006207
Epoch 60/200 - Train Error: 0.007055, Val Error: 0.006194
Epoch 80/200 - Train Error: 0.006991, Val Error: 0.006125
Epoch 100/200 - Train Error: 0.001255, Val Error: 0.001224
Epoch 120/200 - Train Error: 0.000765, Val Error: 0.000784
Epoch 140/200 - Train Error: 0.000663, Val Error: 0.000730
Epoch 160/200 - Train Error: 0.000619, Val Error: 0.000730
Epoch 180/200 - Train Error: 0.000595, Val Error: 0.000731
Epoch 200/200 - Train Error: 0.000577, Val Error: 0.000727

Results:
  RMSE:           $19,873.23
  MAE:            $14,334.09
  R²: 

KeyboardInterrupt: 

In [6]:
combined_results = study_combined_regularization(
    TRAIN_CSV,
    VAL_CSV,
    TEST_CSV,
    activation="sigmoid"
)




STUDY 3: COMBINED L2 + DROPOUT REGULARIZATION

Combining both techniques can leverage their complementary strengths:
  • L2: Penalizes large weights
  • Dropout: Prevents neuron co-adaptation


────────────────────────────────────────────────────────────────────────────────
Testing: Light regularization
  L2 Lambda = 0.001, Dropout Rate = 0.1
────────────────────────────────────────────────────────────────────────────────
Epoch 20/200 - Train Error: 0.007116, Val Error: 0.006135
Epoch 40/200 - Train Error: 0.007184, Val Error: 0.006134
Epoch 60/200 - Train Error: 0.007137, Val Error: 0.006134
Epoch 80/200 - Train Error: 0.007083, Val Error: 0.006121
Epoch 100/200 - Train Error: 0.007147, Val Error: 0.006068
Epoch 120/200 - Train Error: 0.006791, Val Error: 0.005795
Epoch 140/200 - Train Error: 0.005326, Val Error: 0.004250
Epoch 160/200 - Train Error: 0.003042, Val Error: 0.002048
Epoch 180/200 - Train Error: 0.003062, Val Error: 0.002337
Epoch 200/200 - Train Error: 0.002952, Val Er